In [ ]:
#@title #**1. Install The Required Softwares**
#@markdown Execute this cell first to download and install third-party softwares

import os, re, shutil, tqdm.notebook, subprocess, pandas as pd, numpy as np, copy
from google.colab import files
from xml.dom import minidom
from IPython.utils import io
from IPython.display import clear_output

os.environ['GPU_INCLUDE_PATH'] = '/usr/local/cuda/include'
os.environ['GPU_LIBRARY_PATH'] = '/usr/local/cuda/lib64'

TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'
try:
  with tqdm.notebook.tqdm(total = 100, bar_format = TQDM_BAR_FORMAT) as pbar:
    with io.capture_output() as captured:
      %shell wget https://autodock.scripps.edu/wp-content/uploads/sites/56/2021/10/autodocksuite-4.2.6-x86_64Linux2.tar
      %shell tar -xvf '/content/autodocksuite-4.2.6-x86_64Linux2.tar' -C '/content'
      pbar.update(15)

      %shell pip install zipfile36
      from zipfile36 import ZipFile
      pbar.update(15)

      %shell wget https://github.com/ccsb-scripps/AutoDock-GPU/archive/refs/heads/develop.zip -O autodock-gpu.zip
      %shell unzip '/content/autodock-gpu.zip'
      os.chdir('AutoDock-GPU-develop')
      %shell make DEVICE=CUDA
      pbar.update(20)

      %cd /content
      %shell wget -c https://ccsb.scripps.edu/download/532/ -O mgltools.tar.gz
      %shell mkdir mgltools
      %shell tar -xvzf mgltools.tar.gz -C mgltools --strip-components=1

      %shell wget https://ccsb.scripps.edu/adfr/download/1038/ -O adfr.tar.gz
      %shell tar -xf adfr.tar.gz
      %shell mkdir adfr
      %cd /content/ADFRsuite_x86_64Linux_1.0
      %shell yes | ./install.sh -d /content/adfr
      %cd /content
      pbar.update(30)

      %cd mgltools
      %shell ./install.sh
      pbar.update(10)

      %shell pip install ipywidgets meeko
      from ipywidgets import widgets
      %shell pip install rdkit
      from rdkit import Chem
      from rdkit.Chem import AllChem
      pbar.update(10)
except subprocess.CalledProcessError:
  print(captured)
  raise

working_dir = os.path.join('/content', 'working_dir')
os.mkdir(working_dir)
os.chdir(working_dir)

  0%|          | 0/100 [elapsed: 00:00 remaining: ?]

In [ ]:
#@title #**2. Upload Receptor and Ligand File**
#@markdown Please upload the **receptor (PDB)** and **ligand (SDF)** simultaneously, then select what file being a receptor or a ligand
#@markdown **Note:** The file name should not contain any space

working_dir = os.path.join('/content', 'working_dir')
if (not os.path.isdir(working_dir)):
  os.mkdir(working_dir)
else:
  shutil.rmtree(working_dir)
  os.mkdir(working_dir)
os.chdir(working_dir)

receptor_name = ''
ligand_name = ''

uploaded_files = list(files.upload().keys())
clear_output()

receptor_input = widgets.Dropdown(
    options = uploaded_files,
    description = 'Receptor:',
    layout = widgets.Layout(margin = '0 0 20px 0')
)
display(receptor_input)

ligand_input = widgets.Dropdown(
    options = uploaded_files,
    description = 'Ligand:'
)
display(ligand_input)

button = widgets.Button(
    description = 'Select',
    layout = widgets.Layout(margin = '20px 0 20px 90px')
)
display(button)

receptor_label = widgets.Label(value = 'Receptor:')
display(receptor_label)
ligand_label = widgets.Label(value = 'Ligand:')
display(ligand_label)

def incorporate_names(b):
  global receptor_name, ligand_name
  receptor_name = receptor_input.value
  ligand_name = ligand_input.value
  receptor_label.value = 'Receptor: ' + receptor_name
  ligand_label.value = 'Ligand: ' + ligand_name

button.on_click(incorporate_names)

Dropdown(description='Receptor:', layout=Layout(margin='0 0 20px 0'), options=('ligand.sdf', 'receptor.pdb'), …

Dropdown(description='Ligand:', options=('ligand.sdf', 'receptor.pdb'), value='ligand.sdf')

Button(description='Select', layout=Layout(margin='20px 0 20px 90px'), style=ButtonStyle())

Label(value='Receptor:')

Label(value='Ligand:')

In [ ]:
#@title #**3. Convert Receptor and Ligand File to PDBQT Format**

os.chdir(working_dir)

for f in os.listdir():
  if((f != receptor_name) and (f != ligand_name) and (not os.path.isdir(f))):
    os.remove(f)

receptor_pdbqt = re.split('\.', receptor_name)[0] + '.pdbqt'
ligand_pdbqt = re.split('\.', ligand_name)[0] + '.pdbqt'

print(f'Convert {receptor_name} to PDBQT format')
res_receptor = os.system(f'/content/adfr/bin/prepare_receptor -r {receptor_name}')

print(f'Convert {ligand_name} to PDBQT format')
res_ligand = os.system(f'mk_prepare_ligand.py -i {ligand_name} -o {ligand_pdbqt}')

Convert receptor.pdb to PDBQT format
Convert ligand.sdf to PDBQT format


In [ ]:
#@title #**4. Find Ligand Center Point**
#@markdown Center point of x, y, z dimension will be used in the docking process.
#@markdown Once this cell has been executed, the center point of the ligand will be displayed as center_x, center_y, and center_z

os.chdir(working_dir)

supplier = Chem.SDMolSupplier(ligand_name, removeHs=False)
mol = supplier[0]
conf = mol.GetConformer()

coords = np.array([list(conf.GetAtomPosition(i)) for i in range(mol.GetNumAtoms())])
center_x, center_y, center_z = coords.mean(axis=0)

print(f'Center of coordinates: x={center_x}, y={center_y}, z={center_z}')

Center of coordinates: x=11.07394411764706, y=13.791338235294116, z=17.3417


In [ ]:
#@title #**5. Specify Gridbox Dimension**
#@markdown Specify the minimum and maximum size of the gridbox, and the stride

gridbox_min = 15 #@param {type:'integer'}
gridbox_max = 65 #@param {type:'integer'}
stride = 1 #@param {type:'integer'}

In [ ]:
#@title #**5. Generate GPF file for each ligand**
#@markdown This will generate GPF folder contains different folders for each gridbox size

gpf_dir = os.path.join(working_dir, 'gpf')
if (not os.path.isdir(gpf_dir)):
  os.mkdir(gpf_dir)
else:
  shutil.rmtree(gpf_dir)
  os.mkdir(gpf_dir)

for size in range(gridbox_min, gridbox_max + 1, stride):
  cld = os.path.join(gpf_dir, str(size))
  os.mkdir(cld)

  shutil.copyfile(f'./{ligand_pdbqt}', f'{cld}/{ligand_pdbqt}')
  shutil.copyfile(f'./{receptor_pdbqt}', f'{cld}/{receptor_pdbqt}')

  os.chdir(cld) # Move to current ligand directory

  npts = f'npts=\'{size},{size},{size}\''
  gridcenter = f'gridcenter=\'{center_x},{center_y},{center_z}\''
  os.system(f'/content/mgltools/bin/pythonsh /content/mgltools/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_gpf4.py -l {ligand_pdbqt} -r {receptor_pdbqt} -p {npts} -p {gridcenter}')

  os.chdir(working_dir) # Move back to working directory

In [ ]:
#@title #**6. Run Autogrid4**
#@markdown Autogrid4 will generate all files needed for docking step

receptor_noext = receptor_name.split('.')[0]
gpf_file = receptor_noext + '.gpf'
gpf_log = receptor_noext + '.glg'

for lf in os.listdir(gpf_dir):
  os.chdir(f'{gpf_dir}/{lf}') # Move to individual ligand folder

  subprocess.run(['/content/x86_64Linux2/autogrid4', '-p', gpf_file, '-l', gpf_log])

os.chdir(working_dir) # Move back to working directory
clear_output()

In [ ]:
#@title #**7. Run AutoDock-GPU**
#@markdown Run this cell to start gridbox validation

result_dir = os.path.join(working_dir, 'result')
if (not os.path.isdir(result_dir)):
  os.mkdir(result_dir)
else:
  shutil.rmtree(result_dir)
  os.mkdir(result_dir)

for size in range(gridbox_min, gridbox_max + 1, stride):
  cld = os.path.join(result_dir, str(size))
  os.mkdir(cld)

recap = []

for size in os.listdir(gpf_dir):
  result = subprocess.run([
      '/content/AutoDock-GPU-develop/bin/autodock_gpu_128wi',
      '--ffile', f'{gpf_dir}/{size}/{receptor_noext}.maps.fld',
      '--lfile', f'{gpf_dir}/{size}/{ligand_pdbqt}',
      '--resnam', 'output',
      '--xmloutput', '1',
      '--gbest', '1',
      '--nrun', '100',
      '--psize', '100',
      # '--gfpop', '1',
      # '--npdb', '1',
      '--derivtype', '-T'
    ], capture_output=True, text=True)

  if (os.path.exists(f'{working_dir}/output-best.pdbqt')):
    naming = f'gridbox_x{size}_y{size}_z{size}'

    shutil.move('output-best.pdbqt', f'{result_dir}/{size}/{naming}.pdbqt')
    shutil.move('output.dlg', f'{result_dir}/{size}/{naming}.dlg')
    shutil.move('output.xml', f'{result_dir}/{size}/{naming}.xml')

    os.system(f'mk_export.py {result_dir}/{size}/{naming}.pdbqt -s {result_dir}/{size}/{naming}.sdf')

    match = re.search(r"best inter \+ intra\s+(-?\d+\.\d+)\s+kcal/mol", result.stdout, re.IGNORECASE)
    recap.append([f'{size}', float(match.group(1))])

    print(f'Succeed running for gridbox {size}^3')
  else:
    print(f'Cannot running for gridbox {size}^3')

# clear_output()

Succeed running for gridbox 59^3
Succeed running for gridbox 29^3
Succeed running for gridbox 19^3
Succeed running for gridbox 64^3
Succeed running for gridbox 27^3
Succeed running for gridbox 24^3
Succeed running for gridbox 30^3
Succeed running for gridbox 31^3
Succeed running for gridbox 58^3
Succeed running for gridbox 17^3
Succeed running for gridbox 47^3
Succeed running for gridbox 45^3
Succeed running for gridbox 22^3
Succeed running for gridbox 51^3
Succeed running for gridbox 50^3
Succeed running for gridbox 37^3
Succeed running for gridbox 20^3
Succeed running for gridbox 55^3
Succeed running for gridbox 35^3
Succeed running for gridbox 53^3
Succeed running for gridbox 39^3
Succeed running for gridbox 38^3
Succeed running for gridbox 23^3
Succeed running for gridbox 18^3
Succeed running for gridbox 25^3
Succeed running for gridbox 15^3
Succeed running for gridbox 56^3
Succeed running for gridbox 41^3
Succeed running for gridbox 26^3
Succeed running for gridbox 34^3
Succeed ru

In [ ]:
#@title #**8. Calculate RMSD**
#@markdown After running this cell, there would be zip-formatted archive contains
#@markdown files that automatically will be downloaded

new_recap = copy.deepcopy(recap)

for idx, gridbox in enumerate(recap):
  size = gridbox[0]
  sdf = f'gridbox_x{size}_y{size}_z{size}.sdf'
  shutil.copyfile(f'{result_dir}/{size}/{sdf}', f'{sdf}')

  mol1 = Chem.SDMolSupplier(ligand_name, removeHs=False)[0]
  mol2 = Chem.SDMolSupplier(sdf, removeHs=False)[0]

  rmsd = AllChem.GetBestRMS(mol1, mol2)
  new_recap[idx].append(f'{rmsd:.4f}')

  os.remove(sdf)

recap_df = pd.DataFrame(new_recap, columns=['Gridbox Size (Å^3)', 'Binding Energy (kcal/mol)', 'RMSD (Å)']).sort_values(by=['RMSD (Å)'])
recap_df.to_csv('RMSD evaluation for each gridbox size.csv', index=False)

!zip -r gridbox_validation_autodock_gpu.zip ./*
files.download(os.path.join(working_dir, 'gridbox_validation_autodock_gpu.zip'))

  adding: gpf/ (stored 0%)
  adding: gpf/59/ (stored 0%)
  adding: gpf/59/receptor.gpf (deflated 64%)
  adding: gpf/59/receptor.maps.fld (deflated 65%)
  adding: gpf/59/receptor.C.map (deflated 57%)
  adding: gpf/59/receptor.d.map (deflated 71%)
  adding: gpf/59/receptor.OA.map (deflated 58%)
  adding: gpf/59/ligand.pdbqt (deflated 70%)
  adding: gpf/59/receptor.pdbqt (deflated 73%)
  adding: gpf/59/receptor.glg (deflated 85%)
  adding: gpf/59/receptor.NA.map (deflated 57%)
  adding: gpf/59/receptor.N.map (deflated 57%)
  adding: gpf/59/receptor.e.map (deflated 67%)
  adding: gpf/59/receptor.maps.xyz (deflated 3%)
  adding: gpf/59/receptor.A.map (deflated 57%)
  adding: gpf/59/receptor.HD.map (deflated 63%)
  adding: gpf/29/ (stored 0%)
  adding: gpf/29/receptor.gpf (deflated 64%)
  adding: gpf/29/receptor.maps.fld (deflated 65%)
  adding: gpf/29/receptor.C.map (deflated 56%)
  adding: gpf/29/receptor.d.map (deflated 72%)
  adding: gpf/29/receptor.OA.map (deflated 57%)
  adding: gpf/29

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>